In [1]:
from surprise import Dataset, Reader, SVD
from surprise.model_selection import train_test_split
from surprise import accuracy

In [9]:
# import streamlit as st
import pandas as pd
import zipfile

# Extract the CSV file from the zip archive
with zipfile.ZipFile('clean_dataset.zip', 'r') as zipf:
    zipf.extractall()

# Load the clean DataFrame
df_clean = pd.read_csv('clean_dataset.csv')



In [10]:
df = df_clean

In [3]:
df_clean.head()

fit  user_id bust_size  item_id  weight  rating     rented_for  \
0  fit   420272       34d  2260466   137.0       5       vacation   
1  fit   273551       34b   153475   132.0       5          other   
2  fit   909926       34c   126335   135.0       4  formal affair   
3  fit   151944       34b   616682   145.0       5        wedding   
4  fit   734848       32b   364092   138.0       4           date   

                                         review_text          body_type  \
0  An adorable romper! Belt and zipper were a lit...          hourglass   
1  I rented this dress for a photo shoot. The the...  straight & narrow   
2  I rented this for my company's black tie award...               pear   
3  I have always been petite in my upper body and...           athletic   
4  Didn't actually wear it. It fit perfectly. The...           athletic   

                                     review_summary category  height  size  \
0                              So many compliments!   romper  172.72    14   
1                           I felt so glamourous!!!     gown  167.64    12   
2  Dress arrived on time and in perfect condition.     dress  165.10     8   
3                   Was in love with this dress !!!     gown  175.26    12   
4                   Traditional with a touch a sass    dress  172.72     8   

    age         review_date  \
0  28.0      April 20, 2016   
1  36.0       June 18, 2013   
2  34.0   February 12, 2014   
3  27.0  September 26, 2016   
4  45.0      April 30, 2016   

                                    processed_review  \
0  ['ador', 'romper', 'belt', 'zipper', 'littl', ...   
1  ['rent', 'dress', 'photo', 'shoot', 'theme', '...   
2  ['rent', 'compani', 'black', 'tie', 'award', '...   
3  ['alway', 'petit', 'upper', 'bodi', 'extrem', ...   
4  ['actual', 'wear', 'fit', 'perfectli', 'fabric...   

                                 processed_summary  sentiment_score  \
0                           ['mani', 'compliment']         0.625000   
1                              ['felt', 'glamour']         0.000000   
2  ['dress', 'arriv', 'time', 'perfect', 'condit']         1.000000   
3                                ['love', 'dress']         0.976562   
4                      ['tradit', 'touch', 'sass']         0.000000   

   hybrid_rating  
0       5.625000  
1       5.000000  
2       5.000000  
3       5.976562  
4       4.000000

In [1]:
import streamlit as st
from surprise import SVD, Dataset, Reader
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import TruncatedSVD
from sklearn.metrics.pairwise import cosine_similarity
import pandas as pd
import numpy as np
import joblib

In [18]:
# import streamlit as st
# from surprise import SVD, Dataset, Reader
# from sklearn.feature_extraction.text import TfidfVectorizer
# from sklearn.decomposition import TruncatedSVD
# from sklearn.metrics.pairwise import cosine_similarity
# import pandas as pd
# import numpy as np
# import joblib

# # Load SVD model
# final_svd_model = SVD(n_epochs=10, lr_all=0.01, reg_all=0.4)

# # Load content-based model
# def train_content_based_model(df):
#     sample_size = 20000
#     df_sample = df.sample(n=sample_size, random_state=42)

#     # Combine relevant text features into a single column for content-based filtering
#     df_sample['content'] = df_sample['review_summary'].fillna('') + ' ' + df_sample['category'].fillna('') + ' ' \
#                            + df_sample['body_type'].fillna('') + ' ' + df_sample['rented_for'].fillna('')

#     # Create a TF-IDF vectorizer
#     tfidf_vectorizer = TfidfVectorizer(stop_words='english')

#     # Fit and transform the content column
#     tfidf_matrix = tfidf_vectorizer.fit_transform(df_sample['content'])

#     # Implement batch processing for cosine similarity
#     def calculate_cosine_similarity_in_batches(matrix, batch_size=1000):
#         num_items = matrix.shape[0]
#         cosine_sim_batches = []

#         for i in range(0, num_items, batch_size):
#             start_idx = i
#             end_idx = min((i + batch_size), num_items)
#             batch_matrix = matrix[start_idx:end_idx, :]
#             cosine_sim_batch = cosine_similarity(batch_matrix, matrix, dense_output=False)
#             cosine_sim_batches.append(cosine_sim_batch)

#         return np.vstack(cosine_sim_batches)

#     # Calculate cosine similarity in batches
#     cosine_sim = calculate_cosine_similarity_in_batches(tfidf_matrix)

#     # Apply dimensionality reduction using TruncatedSVD
#     num_components = 100
#     svd = TruncatedSVD(n_components=num_components)
#     tfidf_matrix_reduced = svd.fit_transform(tfidf_matrix)
#     cosine_sim_reduced = cosine_similarity(tfidf_matrix_reduced, dense_output=False)

#     # Save the content-based model
#     joblib.dump((cosine_sim_reduced, df_sample['item_id']), 'content_based_model.pkl')

#     return cosine_sim_reduced, df_sample['item_id']

# # Streamlit App
# st.title("Fashion Recommender System")

# # Sidebar with input options
# st.sidebar.header("User Input Features")

# # Add input options based on your model features (e.g., body type, rented for, etc.)
# user_input = {}
# user_input['Body Type'] = st.sidebar.selectbox('Select Body Type', df_clean['body_type'].unique())
# user_input['Rented For'] = st.sidebar.selectbox('Select Rented For', df_clean['rented_for'].unique())
# # Add more input options as needed...

# # Add a button to trigger recommendations
# if st.sidebar.button('Get Recommendations'):
#     # Load content-based model
#     content_based_model, item_ids = train_content_based_model(df_clean)

#     # Function to get content-based recommendations for a given item after dimensionality reduction
#     def get_content_based_recommendations_reduced(item_id, cosine_sim=content_based_model, df=df_clean, item_ids=item_ids):
#         idx = item_ids[item_ids == item_id].index[0]
#         sim_scores = list(enumerate(cosine_sim[idx]))
#         sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
#         sim_scores = sim_scores[1:11]  # Exclude the item itself and take top 10
#         item_indices = [score[0] for score in sim_scores]
#         return df['item_id'].iloc[item_indices]

#     # Example: Get content-based recommendations for an item
#     item_id_to_recommend_for = 1973037
#     content_based_recommendations_reduced = get_content_based_recommendations_reduced(item_id_to_recommend_for)

#     # Display the recommendations
#     st.write(f"Content-Based Recommendations (Reduced) for Item {item_id_to_recommend_for}:")
#     st.write(content_based_recommendations_reduced)


In [11]:
import pickle

In [12]:
pickle_in = open('als_model',"rb")
als_model = pickle.load(pickle_in)

In [13]:
def main():
    st.title("Fashion Recommender System")

    # Add input elements (e.g., sliders, dropdowns) for user features
    # Example: category, body_type, rented_for
    category = st.selectbox("Select category:", df['category'].unique())
    body_type = st.selectbox("Select body type:", df['body_type'].unique())
    rented_for = st.selectbox("Select rented for:", df['rented_for'].unique())

    # Add a button to trigger recommendations
    if st.button("Get Recommendations"):
        # Call the recommendation function and display the results
        recommendations = get_recommendations(category, body_type, rented_for)
        st.write("Recommendations:")
        st.write(recommendations)

if __name__ == "__main__":
    main()


In [14]:
def get_recommendations(category, body_type, rented_for):
    # Filter the dataset based on user inputs
    filtered_data = df[(df['category'] == category) & (df['body_type'] == body_type) & (df['rented_for'] == rented_for)]

    if filtered_data.empty:
        return "No recommendations available for the selected criteria."

    # Get user-specific recommendations from collaborative filtering
    user_id_to_predict = 420272  # Replace with an actual user ID
    item_ids_to_predict = filtered_data['item_id'].unique()

    predictions_collab = [als_model.predict(user_id_to_predict, item_id) for item_id in item_ids_to_predict]
    predicted_ratings_collab = [(prediction.iid, prediction.est) for prediction in predictions_collab]

    # Sort collaborative filtering predictions by rating in descending order
    sorted_predictions_collab = sorted(predicted_ratings_collab, key=lambda x: x[1], reverse=True)

    # Get content-based recommendations for a sample item (replace with user-specific item)
    sample_item_id = filtered_data['item_id'].iloc[0]
    content_based_recommendations = get_content_based_recommendations_reduced(sample_item_id)

    # Combine collaborative filtering and content-based recommendations
    combined_recommendations = sorted_predictions_collab + content_based_recommendations.tolist()

    # Deduplicate and sort the combined recommendations
    unique_recommendations = list(set(combined_recommendations))
    sorted_recommendations = sorted(unique_recommendations, key=lambda x: x[1], reverse=True)

    # Return the top N recommendations
    top_n = 10
    return sorted_recommendations[:top_n]
